In [12]:
# import libraries needed
import IPython
import pandas as pd
import numpy as np
from statistics import mode
import matplotlib.pyplot as plt  
from sklearn.preprocessing import MinMaxScaler
from IPython.display import display, HTML
from pprint import pprint
import xgboost as xgb

## TODO

- [ ] 清洗column： ID
- [ ] 缺失值处理：先全删除
- [ ] one-hot, number
- [ ] 降维
- [ ] xgboost, RF
- [ ] metric
- [ ] 24 feature： 加起来

In [2]:
df = pd.read_csv('./data/diabetic_data.csv').sort_values(by=['patient_nbr'])
len(df)

101766

In [3]:
print(df.info())
df.describe()
nominals = [
    "race",  # 人种
    "gender",  # 性别
    "age",  # 年龄
    "admission_type_id",  # 入院类型，如紧急，新生（？），未知
    "discharge_disposition_id",  # 出院处置，如回家，到期，未知
    "admission_source_id", # 
    "payer_code",
    "medical_specialty",
    "diag_1",
    "diag_2",
    "diag_3",
    "max_glu_serum",
    "A1Cresult",
    "metformin",
    "repaglinide",
    "nateglinide",
    "chlorpropamide",
    "glimepiride",
    "acetohexamide",
    "glipizide",
    "glyburide",
    "tolbutamide",
    "pioglitazone",
    "rosiglitazone",
    "acarbose",
    "miglitol",
    "troglitazone",
    "tolazamide",
    "examide",
    "citoglipton",
    "insulin",
    "glyburide-metformin",
    "glipizide-metformin",
    "glimepiride-pioglitazone",
    "metformin-rosiglitazone",
    "metformin-pioglitazone",
    "change",
    "diabetesMed",
    "readmitted",
]

<class 'pandas.core.frame.DataFrame'>
Int64Index: 101766 entries, 4267 to 96147
Data columns (total 50 columns):
encounter_id                101766 non-null int64
patient_nbr                 101766 non-null int64
race                        101766 non-null object
gender                      101766 non-null object
age                         101766 non-null object
weight                      101766 non-null object
admission_type_id           101766 non-null int64
discharge_disposition_id    101766 non-null int64
admission_source_id         101766 non-null int64
time_in_hospital            101766 non-null int64
payer_code                  101766 non-null object
medical_specialty           101766 non-null object
num_lab_procedures          101766 non-null int64
num_procedures              101766 non-null int64
num_medications             101766 non-null int64
number_outpatient           101766 non-null int64
number_emergency            101766 non-null int64
number_inpatient            101

In [4]:
def show_crosstab(col):
    grouped = df.groupby([col, 'readmitted']).size().to_frame().reset_index()
    crosstab = pd.crosstab(grouped[col], grouped['readmitted'], values=grouped[0], aggfunc=lambda x: x)
    display(crosstab)

for col in nominals[:-1]:
    show_crosstab(col)

readmitted,<30,>30,NO
race,,,
?,188,538,1547
AfricanAmerican,2155,6634,10421
Asian,65,161,415
Caucasian,8592,27124,40383
Hispanic,212,642,1183
Other,145,446,915


readmitted,<30,>30,NO
gender,,,
Female,6152.0,19518.0,29038.0
Male,5205.0,16027.0,25823.0
Unknown/Invalid,NaN,NaN,3.0


readmitted,<30,>30,NO
age,,,
[0-10),3,26,132
[10-20),40,224,427
[20-30),236,510,911
[30-40),424,1187,2164
[40-50),1027,3278,5380
[50-60),1668,5917,9671
[60-70),2502,7897,12084
[70-80),3069,9475,13524
[80-90),2078,6223,8896


readmitted,<30,>30,NO
admission_type_id,,,
1,6221.0,19309.0,28460.0
2,2066.0,6452.0,9962.0
3,1961.0,5746.0,11162.0
4,1.0,2.0,7.0
5,495.0,1721.0,2569.0
6,586.0,2231.0,2474.0
7,NaN,NaN,21.0
8,27.0,84.0,209.0


readmitted,<30,>30,NO
discharge_disposition_id,,,
1,5602.0,21517.0,33115.0
2,342.0,668.0,1118.0
3,2046.0,4916.0,6992.0
4,104.0,278.0,433.0
5,247.0,350.0,587.0
6,1638.0,5362.0,5902.0
7,90.0,221.0,312.0
8,15.0,38.0,55.0
9,9.0,2.0,10.0


...

readmitted,<30,>30,NO
glimepiride-pioglitazone,,,
No,11357.0,35544.0,54864.0
Steady,NaN,1.0,NaN


readmitted,<30,>30,NO
metformin-rosiglitazone,,,
No,11357.0,35545.0,54862.0
Steady,NaN,NaN,2.0


readmitted,<30,>30,NO
metformin-pioglitazone,,,
No,11357.0,35545.0,54863.0
Steady,NaN,NaN,1.0


readmitted,<30,>30,NO
change,,,
Ch,5558,17272,24181
No,5799,18273,30683


readmitted,<30,>30,NO
diabetesMed,,,
No,2246,7227,13930
Yes,9111,28318,40934


In [5]:
# drop rows

mask = (df['gender'] != 'Unknown/Invalid') & (df['diag_1'] != '?')  # TODO: only consider primary diag
df = df[mask]

# consider only the first encounter for each patient
df = df.drop_duplicates(subset='patient_nbr', keep='first')

# remove all encounters that resulted in either discharge to a hospice or patient death
# to avoid biasing our analysis
df = df[~df['discharge_disposition_id'].isin([11, 13, 14, 19, 20, 21])]

len(df)

69650

In [6]:
# # drop columns

# drop unrelated columns
df = df.drop(columns=["encounter_id", "patient_nbr"])  

# drop columns with too many missing values
df = df.drop(columns=["payer_code", "weight", "medical_specialty"]) 

# drop columns having same value in each row
df = df.drop(columns=["citoglipton", "examide"])

# drop diag_2, diag_3
df = df.drop(columns=["diag_2", "diag_3"])
df = df.drop(columns=["metformin",
    "repaglinide",
    "nateglinide",
    "chlorpropamide",
    "glimepiride",
    "acetohexamide",
    "glipizide",
    "glyburide",
    "tolbutamide",
    "pioglitazone",
    "rosiglitazone",
    "acarbose",
    "miglitol",
    "troglitazone",
    "tolazamide",
    "insulin",
    "glyburide-metformin",
    "glipizide-metformin",
    "glimepiride-pioglitazone",
    "metformin-rosiglitazone",
    "metformin-pioglitazone"
])

In [7]:
# merge ids with same meaning
def merge(df, col, same_ids):
    for ids in same_ids:
        for k in ids[1:]:
            df[col] = df[col].replace(k, ids[0])
    return df

df = merge(df, 'admission_type_id', [
    [1, 2, 7],  # emergence
    [5, 6, 8],  # not avaliable
])
df = merge(df, 'discharge_disposition_id', [
    [18, 25, 26],  # not avaliable
    [1, 6, 8],  # to home
    [2, 3, 4, 5],  # discharge to another hospital
    [10, 12, 15, 16, 17],  # discharge to outpatient
])
df = merge(df, 'admission_source_id', [
    [1, 2, 3], # Referral
    [4, 5, 6, 10, 22, 25], # from another hospital
    [9, 15, 17, 20, 21]  # not avaliable
])

In [8]:
# encoding non-numeric values manully

def encode(df, col, d):
    df[col] = df[col].apply(lambda x: d[x])
    return df

df = encode(df, 'gender', {"Male": 1, "Female": 0})
df = encode(df, 'readmitted', {"NO": 0, ">30": 1, "<30": 2})
df = encode(df, 'change', {'No': 0, 'Ch': 1})
df = encode(df, 'diabetesMed', {'No': 0, 'Yes': 1})
df = encode(df, 'max_glu_serum', {'>300': 2, ">200": 1, 'Norm': 0, 'None': -10})
df = encode(df, 'A1Cresult', {'>8': 2, ">7": 1, 'Norm': 0, 'None': -10})
df['age'] = df['age'].apply(lambda x: int(x[1]))  # encode [0-10) - [90, 100] to 0 - 9

# # encode race
df = encode(df, "race", {k: i for i, k in enumerate(df['race'].unique())})

# encode automaticlly
# for col in ["gender", "readmitted", "change", "diabetesMed", "max_glu_serum", "A1Cresult", "race", "age"]:
#     d = {k: i for i, k in enumerate(df[col].unique())}
#     df[col] = df[col].apply(lambda x: d[x])

In [9]:
# convert diagnosis information: reference from original paper
def in_range(x, bounds=None, sets=None):
    if bounds is None:
        bounds = []
    elif not isinstance(bounds[0], tuple):
        bounds = [bounds]
    
    if sets is None:
        sets = []
    elif not isinstance(sets, tuple):
        sets = [sets]
        
    for (l, r) in bounds:
        if x >= l and x < r:
            return True
    return x in sets

def convert_diag_func(x):
    ranges = {
        ((390, 460), 785),
        ((460, 520), 786),
        ((520, 580), 787),
        (None, 250),
        ((800, 1000), None),
        ((710, 740), None),
        ((580, 630), 788),
        ((140, 240), None),
        ((790, 799), (780, 781, 784)),
        (((240, 250), (251, 280)), None),
        ((680, 710), 782),
        ((1, 140), None),
        ((290, 320), None),
        ((280, 290), None),
        ((320, 360), None),
        ((630, 680), None),
        ((360, 390), None),
        ((740, 760), None),
    }
    if x == '?':
        return -1
    elif isinstance(x, str) and (x.startswith('V') or x.startswith('E')):
        return 0

    for i, (bounds, sets) in enumerate(ranges):
        if in_range(int(float(x)), bounds, sets):
            return i+1
        
df['diag_1'] = df['diag_1'].apply(convert_diag_func)

In [10]:
df.head(16).T

,4267,5827,67608,17494,2270,14180,18234,15848,61382,2279,7866,25911,1083,2001,11049,2484
race,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
gender,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
age,5.0,5.0,8.0,8.0,3.0,6.0,6.0,4.0,7.0,7.0,6.0,7.0,6.0,7.0,8.0,6.0
admission_type_id,1.0,3.0,1.0,1.0,1.0,1.0,3.0,1.0,3.0,1.0,1.0,3.0,1.0,3.0,1.0,1.0
discharge_disposition_id,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,2.0,2.0,1.0
admission_source_id,1.0,1.0,7.0,7.0,7.0,7.0,1.0,1.0,4.0,1.0,1.0,1.0,7.0,1.0,7.0,7.0
time_in_hospital,8.0,2.0,4.0,3.0,5.0,10.0,9.0,2.0,14.0,12.0,8.0,1.0,2.0,7.0,7.0,4.0
num_lab_procedures,77.0,49.0,68.0,46.0,49.0,54.0,52.0,50.0,21.0,47.0,57.0,31.0,15.0,27.0,77.0,47.0
num_procedures,6.0,1.0,2.0,0.0,0.0,2.0,1.0,5.0,0.0,2.0,6.0,1.0,0.0,3.0,0.0,4.0
num_medications,33.0,11.0,23.0,20.0,5.0,19.0,16.0,13.0,15.0,18.0,31.0,9.0,14.0,16.0,12.0,16.0


array([[0., 0., 5., ..., 1., 1., 2.],
       [0., 0., 5., ..., 0., 0., 0.],
       [0., 0., 8., ..., 0., 1., 0.],
       ...,
       [0., 0., 8., ..., 1., 1., 0.],
       [0., 0., 4., ..., 1., 1., 1.],
       [0., 1., 4., ..., 1., 1., 0.]])

# Train and test

In [22]:
from sklearn.model_selection import train_test_split

X = df.values[:, :-1]
y = df.values[:, -1].astype(int)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
xg_train = xgb.DMatrix(X_train, label=y_train)
xg_test = xgb.DMatrix(X_test, label=y_test)

param = {}
param['objective'] = 'multi:softmax'
param['eta'] = 0.1
param['max_depth'] = 6
param['silent'] = 1
param['nthread'] = 4
param['num_class'] = 3

watchlist = [(xg_train, 'train'), (xg_test, 'test')]
num_round = 5
bst = xgb.train(param, xg_train, num_round, watchlist)
# get prediction
pred = bst.predict(xg_test)
error_rate = np.sum(pred != test_Y) / test_Y.shape[0]
print('Test error using softmax = {}'.format(error_rate))

# do the same thing again, but output probabilities
param['objective'] = 'multi:softprob'
bst = xgb.train(param, xg_train, num_round, watchlist)
# Note: this convention has been changed since xgboost-unity
# get prediction, this is in 1D array, need reshape to (ndata, nclass)
pred_prob = bst.predict(xg_test).reshape(test_Y.shape[0], 6)
pred_label = np.argmax(pred_prob, axis=1)
error_rate = np.sum(pred_label != test_Y) / test_Y.shape[0]
print('Test error using softprob = {}'.format(error_rate))